<a href="https://colab.research.google.com/github/GaikwadSandesh/Tensorflow_tutorials/blob/main/Dropout_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Codebasics Tensorflow/sonar_dataset.csv", header=None)
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
105,0.0116,0.0179,0.0449,0.1096,0.1913,0.0924,0.0761,0.1092,0.0757,0.1006,...,0.0163,0.0099,0.0084,0.0270,0.0277,0.0097,0.0054,0.0148,0.0092,M
101,0.0335,0.0134,0.0696,0.1180,0.0348,0.1180,0.1948,0.1607,0.3036,0.4372,...,0.0244,0.0232,0.0093,0.0159,0.0193,0.0032,0.0377,0.0126,0.0156,M
148,0.0712,0.0901,0.1276,0.1497,0.1284,0.1165,0.1285,0.1684,0.1830,0.2127,...,0.0154,0.0156,0.0054,0.0030,0.0048,0.0087,0.0101,0.0095,0.0068,M
27,0.0177,0.0300,0.0288,0.0394,0.0630,0.0526,0.0688,0.0633,0.0624,0.0613,...,0.0102,0.0122,0.0044,0.0075,0.0124,0.0099,0.0057,0.0032,0.0019,R
106,0.0331,0.0423,0.0474,0.0818,0.0835,0.0756,0.0374,0.0961,0.0548,0.0193,...,0.0078,0.0174,0.0176,0.0038,0.0129,0.0066,0.0044,0.0134,0.0092,M


In [5]:
df.shape

(208, 61)

In [10]:
# check for nan values
df.isna().sum().max()

0

In [11]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [12]:
df[60].value_counts() # label is not skewed

M    111
R     97
Name: 60, dtype: int64

In [13]:
X = df.drop(60, axis=1)
y = df[60]
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [14]:
y = pd.get_dummies(y, drop_first=True)
y.sample(5) # R --> 1 and M --> 0

,R
43,1
111,0
180,0
147,0
105,0


In [15]:
y.value_counts()

R
0    111
1     97
dtype: int64

---
Train test split




In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [17]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
67,0.0368,0.0403,0.0317,0.0293,0.0820,0.1342,0.1161,0.0663,0.0155,0.0506,...,0.0058,0.0091,0.0160,0.0160,0.0081,0.0070,0.0135,0.0067,0.0078,0.0068
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
164,0.0163,0.0198,0.0202,0.0386,0.0752,0.1444,0.1487,0.1484,0.2442,0.2822,...,0.0027,0.0077,0.0026,0.0031,0.0083,0.0020,0.0084,0.0108,0.0083,0.0033
179,0.0394,0.0420,0.0446,0.0551,0.0597,0.1416,0.0956,0.0802,0.1618,0.2558,...,0.0118,0.0146,0.0040,0.0114,0.0032,0.0062,0.0101,0.0068,0.0053,0.0087
19,0.0126,0.0149,0.0641,0.1732,0.2565,0.2559,0.2947,0.4110,0.4983,0.5920,...,0.0153,0.0092,0.0035,0.0098,0.0121,0.0006,0.0181,0.0094,0.0116,0.0063


# Models

## Model without Dropout Layer

In [18]:
import tensorflow as tf
from tensorflow import keras

In [19]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.6821 - accuracy: 0.5513
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6474 - accuracy: 0.6795
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6223 - accuracy: 0.6731
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6028 - accuracy: 0.7372
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5644 - accuracy: 0.7564
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5396 - accuracy: 0.7628
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5025 - accuracy: 0.7564
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4753 - accuracy: 0.8077
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4454 - accuracy: 0.8013
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4221 - accuracy: 0.8141
Epoch 11/

In [20]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 8ms/step - loss: 1.1150 - accuracy: 0.7500


[1.1150254011154175, 0.75]

In [21]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

[1.2038130e-10 9.2707658e-01 7.5855601e-01 1.8975014e-06 9.9999630e-01
 9.9981344e-01 6.2596428e-01 9.9999690e-01 1.5887017e-06 9.9999917e-01]
[0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]


In [22]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.81      0.77        27
           1       0.77      0.68      0.72        25

    accuracy                           0.75        52
   macro avg       0.75      0.75      0.75        52
weighted avg       0.75      0.75      0.75        52



##  Model with Dropout Layer

In [27]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.7253 - accuracy: 0.6090
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6944 - accuracy: 0.5000
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7011 - accuracy: 0.5321
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7051 - accuracy: 0.5513
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6929 - accuracy: 0.5641
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5064
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7004 - accuracy: 0.5000
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6908 - accuracy: 0.5705
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6949 - accuracy: 0.5000
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6830 - accuracy: 0.5962
Epoch 11/

In [28]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 6ms/step - loss: 0.4751 - accuracy: 0.7885


[0.47507035732269287, 0.7884615659713745]

In [25]:
#Training Accuracy is still good but Test Accuracy Improved

In [29]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.81      0.77        27
           1       0.77      0.68      0.72        25

    accuracy                           0.75        52
   macro avg       0.75      0.75      0.75        52
weighted avg       0.75      0.75      0.75        52



In [ ]:
#You can see that by using dropout layer test accuracy increased from 0.75 to 0.79